In [274]:
import pandas as pd
import numpy as np

#需要优化的量

#关系矩阵
#ctd
M1=np.zeros((2,18)) #国产
M3=np.zeros((2,18)) #进口
#dts
M2=np.zeros((19,106)) #国产
M4=np.zeros((19,106)) #进口

#仓库状态矩阵
O=np.zeros((20,1))  #是否开仓
Z=np.zeros((20,1))  #是否智能


In [275]:
#参数导入
#国产，对应1
Q_dm=pd.read_csv('data_p/Q_dm.csv')
Q_dm=Q_dm.to_numpy()
Q_dm=Q_dm[:,1]
Q_dm=Q_dm.reshape((1,106))

In [276]:
#港口，对应0
Q_im=pd.read_csv('data_p/Q_im.csv')
Q_im=Q_im.to_numpy()
Q_im=Q_im[:,1]
Q_im=Q_im.reshape((1,106))


In [277]:
L_dts=pd.read_csv('data_p/d_matrix_dts.csv')
L_dts=L_dts.to_numpy()
L_dts=L_dts[1:]
L_dts=L_dts[:,2:]
L_dts_1=np.delete(L_dts,6,axis=0)#im
L_dts_2=np.delete(L_dts,11,axis=0)#dm
L_dts_1.shape

(19, 106)

In [278]:
T_dts=pd.read_csv('data_p/t_matrix_dts.csv')
T_dts=T_dts.to_numpy()
T_dts=T_dts[1:]
T_dts=T_dts[:,2:]
T_dts_1=np.delete(T_dts,6,axis=0)#im
T_dts_2=np.delete(T_dts,11,axis=0)#dm
T_dts_1

array([[532.0, 167.0, 491.0, ..., 578.0, 1478.0, 364.0],
       [1376.0, 959.0, 1295.0, ..., 556.0, 644.0, 935.0],
       [1639.0, 1209.0, 1163.0, ..., 962.0, 1130.0, 1253.0],
       ...,
       [1169.0, 758.0, 1140.0, ..., 310.0, 877.0, 709.0],
       [1189.0, 753.0, 651.0, ..., 501.0, 1249.0, 816.0],
       [948.0, 406.0, 859.0, ..., 175.0, 1137.0, 440.0]], dtype=object)

In [279]:
L_ctd=pd.read_csv('data_p/d_matrix_ctd.csv')
L_ctd=L_ctd.to_numpy()
L_ctd=L_ctd[:,2:]
L_ctd

array([[2071.0, 889.0, 2055.0, 1753.0, 2805.0, 598.0, 648.0, 3311.0,
        934.0, 1160.0, 834.0, 1694.0, 1028.0, 577.0, 1997.0, 1031.0,
        1820.0, 1532.0],
       [57.0, 1461.0, 1841.0, 1752.0, 837.0, 2131.0, 2122.0, 1346.0,
        1276.0, 999.0, 1327.0, 651.0, 1154.0, 2147.0, 91.0, 1132.0,
        1071.0, 698.0]], dtype=object)

In [280]:
T_ctd=pd.read_csv('data_p/t_matrix_ctd.csv')
T_ctd=T_ctd.to_numpy()
T_ctd=T_ctd[:,2:]
T_ctd

array([[1332.0, 649.0, 1349.0, 1209.0, 1732.0, 421.0, 449.0, 2024.0,
        696.0, 852.0, 624.0, 1148.0, 801.0, 404.0, 1298.0, 757.0, 1212.0,
        1032.0],
       [110.0, 1026.0, 1279.0, 1247.0, 682.0, 1386.0, 1391.0, 1003.0,
        949.0, 738.0, 976.0, 491.0, 885.0, 1398.0, 115.0, 832.0, 813.0,
        496.0]], dtype=object)

In [281]:
Pf=pd.read_csv('data_p/Pf.csv')
Pf=Pf.to_numpy()
Pf=Pf[:,1:]
Pf

array([[0.0073  , 0.00365 ],
       [0.00635 , 0.003175],
       [0.0067  , 0.00335 ],
       [0.0068  , 0.0034  ],
       [0.007   , 0.0035  ],
       [0.007   , 0.0035  ],
       [0.0075  , 0.00375 ],
       [0.0075  , 0.00375 ],
       [0.0075  , 0.00375 ],
       [0.0072  , 0.0036  ],
       [0.007   , 0.0035  ],
       [0.0065  , 0.00325 ],
       [0.0075  , 0.00375 ],
       [0.007   , 0.0035  ],
       [0.0072  , 0.0036  ],
       [0.0075  , 0.00375 ],
       [0.0068  , 0.0034  ],
       [0.0069  , 0.00345 ],
       [0.0065  , 0.00325 ],
       [0.007   , 0.0035  ]])

In [282]:
Of=pd.read_csv('data_p/Of.csv')
Of=Of.to_numpy()
Of=Of[:,1]
Of

array([25., 25., 25., 25., 25., 25.,  0., 25., 25., 25., 25.,  0., 25.,
       25., 25., 25., 25., 25., 25., 25.])

In [283]:
Cap=pd.read_csv('data_p/Cap.csv')
Cap=Cap.to_numpy()
Cap=Cap[:,1]
Cap

array([ 1200.,  1200.,  2000.,  2200.,  1140.,  1300.,  8000.,  2500.,
        1050.,  1500.,  2500., 10000.,  2500.,  1000.,  3500.,  1800.,
        3000.,   800.,  1200.,  1450.])

In [284]:
#约束条件
##时间
Q_all=Q_im.sum()
def time_point(M_im,M_dm,Q_im,Q_dm):
    Q_out=0
    tmp1=M_im*T_dts
    tmp2=M_dm*T_dts
    print(tmp1,"\n",tmp2)
    for i in (0,106):
        if tmp1.sum(axis=1)>=10:
            Q_out+=Q_im[i]
        if tmp2.sum(axis=1)>=10:
            Q_out+=Q_dm[i]
    out_point=Q_out/Q_all
    return out_point



def time_lim(M_im,M_dm,Q_im,Q_dm):
    if time_point(M_im,M_dm,Q_im,Q_dm)<=0.95:
        return True
    return False

#最大订单量不超过最大承载量
def cap_lim(M_im,M_dm):
    tmp=M_im.T*Q_im+M_dm.T*Q_dm
    tmp1=tmp.sum(axis=1)
    for i,j in tmp1,Cap:
        if i>j:
            return False
    return True

#cdc必须开
def cdc_op(O):
    return O[6]==1 and O[11]==1



In [285]:
#目标优化式
##处置成本
def ccl_cp(O,Z,Q,M_s,M_d,P):
    cp=np.zeros((1,20))
    tmp=M_s*Q
    tmp1=tmp.sum(axis=1)
    tmp1=tmp1.reshape((1,19))
    tmp2=M_d*tmp1
    ccp=tmp2.sum()
    for i in (0,20):
        fl=Z[i]
        cp[i]=fl+O[i]+tmp1[i]*P[i,fl]
    return ccl_cp.sum(axis=0)+ccp
#运输成本
def ccl_ct(Q,M_s,M_d,L_s,L_d):
    ct1=np.zeros((1,20))
    sum=0
    tmp=M_s*Q
    tmp1=tmp*L_s.T*1.25
    tmp2=M_d*tmp*L_d.T*0.6
    sum+=tmp1.sum()+tmp2.sum()
    return sum

#总成本
def cal_all(M1,M2,M3,M4,O,Z,):
    c=0
    #进口
    c+=ccl_cp(O,Z,Q_im,M1,M2,Pf)+ccl_ct(Q_im,M1,M2,L_dts_1,L_ctd[0])
    #国产
    c+=ccl_cp(O,Z,Q_dm,M3,M4,Pf)+ccl_ct(Q_dm,M3,M4,L_dts_2,L_ctd[1])
    return c
    
    






In [286]:
def test():
    M1t=np.random.randint(0,2,size=(19,106))
    M2t=np.random.randint(0,2,size=(2,18))
    M3t=np.random.randint(0,2,size=(19,106))
    M4t=np.random.randint(0,2,size=(2,18))
    Ot=np.random.randint(0,2,size=(20,1))
    Zt=np.random.randint(0,2,size=(20,1))
    a=cal_all(M1t,M2t,M3t,M4t,Ot,Zt)
    print(a)

test()


ValueError: operands could not be broadcast together with shapes (2,18) (1,19) 

In [ ]:
from gurobipy import *
m=Model("ass_mov")
m.addVars(M1,M2,M3,M4,O,Z,vtype=GRB.BINARY)
m.update()
m.setObjective(cal_all(M1,M2,M3,M4,O,Z),GRB.MINIMIZE)
m.addConstrs(time_lim(M1,M3,Q_im,Q_dm))
m.addConstrs(cap_lim(M1,M3))
m.addConstrs(cdc_op(O))
m.optimize()
print("M1:",M1,"M2",M2)

TypeError: unhashable type: 'numpy.ndarray'